# <a>Gradient Boosting com LightGBM</a>



---

O Let's Data produz conteúdos gratuitos sobre Data Science!

Confira algumas das nossa produções:

* [Livro Digital - Como se tornar um Cientista de Dados?](https://ebook.letsdata.ai/)
* [Inscreva-se no canal no YouTube](https://www.youtube.com/c/LetsDataAI?sub_confirmation=1)
* [Siga-nos no Instagram](https://www.instagram.com/letsdata.ai)
* Ouça nosso Podcast no [Spotify](https://open.spotify.com/show/0VsNN95jsJVRS424eCFDlg?si=EvQ1KBq1SqGVD4ojHrx4eQ), [Google Podcast](https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5idXp6c3Byb3V0LmNvbS8xNzMxMjUyLnJzcw), [Apple Podcasts](https://podcasts.apple.com/us/podcast/lets-data-podcast/id1559072194), [Castbox](https://castbox.fm/vic/1559072194), [Deezer](https://www.deezer.com/show/2770422) e demais players. 
* Também estamos no [LinkedIn](https://linkedin.com/company/lets-data), [Twitter](https://twitter.com/letsdataAI), [Facebook](https://www.facebook.com/letsdataAI/).
* [Pra finalizar, nosso blog no Medium](https://medium.com/lets-data).

---

## Vamos pro conteúdo!

Queridinhos dos Kaggle Grandmasters, os algoritmos de gradient boosting dominam as aplicações práticas quando o assunto é machine learning para dados tabulares. Hoje veremos uma implementação muito utilizada, LightGBM. Arquirrival do XGBoost e Catboost, o LightGBM promete muita velocidade no treinamento e inferência, sem perder robustez.

Nessa demonstração vamos usar a API do LightGBM para scikit-learn. Existe uma API "nativa" que possui objetos e métodos diferentes, mas se estamos acostumados com os velhos fit, predict, cross_val's da vida, por que não utilizar?

Caso queria aprender a API "original" segue o [link](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html#cv)

Lembrando que precisamos instalar o pacote, já que não vem com o scikit learn! Vamos nessa!

In [1]:
#!pip install lightgbm
#!pip install notebook pandas seaborn scikit-learn

In [2]:
# Importando o lightgbm
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

## <a> Carregando os dados para regressão e classificação </a> 

O foco vai ser em como utilizar o LightGBM, mais que nos problemas de regressão/classificação em si

In [3]:
import seaborn as sns

df_iris = sns.load_dataset('iris')

df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
df_iris.shape

(150, 5)

In [5]:
df_iris['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

## <a> Separando as bases </a>

O foco é LightGBM, mas um splitzinho tem que rolar sempre!

In [6]:
# Separando X e y
X_iris = df_iris.drop(['species'], axis=1)
y_iris = df_iris['species']

In [7]:
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(X_iris, y_iris, random_state=42)

## <a> Criando o objeto com o classificador LightGBM </a>

Veja o quanto é difícil #sqn

In [8]:
classificador_lgbm = lgb.LGBMClassifier()

In [9]:
type(classificador_lgbm)

lightgbm.sklearn.LGBMClassifier

### <a> Compatibilidade com scikit-learn </a>

Olha que massa! O [LGBMClassifier](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html) é uma subclasse de *sklearn.base.ClassifierMixin*, ou seja, é compatível com sklearn. Podemos então utilizar o arcabouço do scikit learn, como métodos de CV, pipelines, etc...

In [10]:
from sklearn.model_selection import cross_val_score

# Funciona com scikit learn! <3
100 * cross_val_score(classificador_lgbm, X_iris_train, y_iris_train).mean()

92.84584980237153

### <a> Pequena tunagem! </a>

O LightGBM permite alteração de vários hiperparâmetros, como learning rate, altura máxima das árvores, quantidade máxima de folhas nas árvores, número de árvores (estimadores), etc...

Nosso foco não é em tunagem de hiperparâmetros, mas vamos mudar somente a altura máxima.

Vamos aproveitar também para mostrar uma característica interessante do LightGBM. Ele permite alterar a implementação do classificador! Podemos, por exemplo, utilizar random forest (bagging) ao invés de algoritmos de boosting! Vamos ver se melhora o resultado.

In [11]:
classificador_lgbm_tunado = lgb.LGBMClassifier(max_depth=2)

100 * cross_val_score(classificador_lgbm_tunado, X_iris_train, y_iris_train).mean()

93.71541501976284

In [12]:
# Classificar com random forest, com fração de 80% da base de treinamento (bagging_fraction)
# e pegando uma nova amostra de 80% a cada iteração (bagging_freq)
classificador_lgbm_rf = lgb.LGBMClassifier(boosting_type='rf', bagging_freq=1, bagging_fraction=0.8)

100 * cross_val_score(classificador_lgbm_rf, X_iris_train, y_iris_train).mean()

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagg

92.80632411067194

### <a> Tipos de classificador </a>

Direto da [documentação](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html): ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest.

In [13]:
classificador_lgbm_dart = lgb.LGBMClassifier(boosting_type='dart')

100 * cross_val_score(classificador_lgbm_dart, X_iris_train, y_iris_train).mean()

94.62450592885375

In [14]:
# Capricho de legibilidade
classificador_campeao = classificador_lgbm_dart

In [15]:
# Com o melhor modelo, podemos utilizar a base toda de treino
classificador_campeao.fit(X_iris_train, y_iris_train)

LGBMClassifier(boosting_type='dart')

In [16]:
# Podemos realizar a predição da base de teste!
predicoes_iris = classificador_campeao.predict(X_iris_test)

predicoes_iris[:10]

array(['versicolor', 'setosa', 'virginica', 'versicolor', 'versicolor',
       'setosa', 'versicolor', 'virginica', 'versicolor', 'versicolor'],
      dtype=object)

In [17]:
y_iris_test

73     versicolor
18         setosa
118     virginica
78     versicolor
76     versicolor
31         setosa
64     versicolor
141     virginica
68     versicolor
82     versicolor
110     virginica
12         setosa
36         setosa
9          setosa
19         setosa
56     versicolor
104     virginica
69     versicolor
55     versicolor
132     virginica
29         setosa
127     virginica
26         setosa
128     virginica
131     virginica
145     virginica
108     virginica
143     virginica
45         setosa
30         setosa
22         setosa
15         setosa
65     versicolor
11         setosa
42         setosa
146     virginica
51     versicolor
27         setosa
Name: species, dtype: object

In [18]:
# Calculando o número de acertos
(predicoes_iris == y_iris_test).sum()

38

In [19]:
# Mas qual o tamanho da base de teste?
len(y_iris_test)

38

In [20]:
acertos = (predicoes_iris == y_iris_test).sum()
total = len(y_iris_test)

acuracia = 100 * acertos / total

acuracia

100.0

## <a> Vamos de Regressão? </a>

Bem rapidinho porque é praticamente a mesma coisa, mudando só o objeto de LGBMClassifier para LGBMRegressor

In [21]:
# Vamos tentar prever o consumo dos carros a partir de suas features
df_mpg = sns.load_dataset('mpg')

df_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [22]:
df_mpg.shape

(398, 9)

In [23]:
df_mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [24]:
# Leon é preguiçoso e tirou origin e name
X_mpg = df_mpg.drop(['mpg', 'origin', 'name'], axis=1)
y_mpg = df_mpg['mpg']

In [25]:
X_mpg_train, X_mpg_test, y_mpg_train, y_mpg_test = train_test_split(X_mpg, y_mpg, random_state=42)

In [26]:
from sklearn import metrics

metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [27]:
# Bem facinho, só mudar de LGBMClassifier para LGBMRegressor
regressor_lgbm = lgb.LGBMRegressor()

cross_val_score(regressor_lgbm, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.0602968789895

In [28]:
regressor_lgbm_tunado = lgb.LGBMRegressor(max_depth=2)

cross_val_score(regressor_lgbm_tunado, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.085218235455716

In [29]:
regressor_lgbm_dart = lgb.LGBMRegressor(boosting_type='dart')

cross_val_score(regressor_lgbm_dart, X_mpg_train, y_mpg_train, scoring='neg_root_mean_squared_error').mean()

-3.8958190536698334

In [30]:
regressor_campeao = regressor_lgbm

In [31]:
regressor_campeao.fit(X_mpg_train,y_mpg_train, eval_metric='root_mean_squared_error')

LGBMRegressor()

In [32]:
predicoes_mpg = regressor_campeao.predict(X_mpg_test)

predicoes_mpg[:10]

array([32.62089047, 30.47819635, 21.1453571 , 15.63407173, 13.12103143,
       24.96860052, 25.72020663, 12.72353355, 17.82671692, 19.06759737])

In [33]:
y_mpg_test[:10]

198    33.0
396    28.0
33     19.0
208    13.0
93     14.0
84     27.0
373    24.0
94     13.0
222    17.0
126    21.0
Name: mpg, dtype: float64

In [34]:
from sklearn.metrics import mean_squared_error
import math

mse = mean_squared_error(y_mpg_test, predicoes_mpg)

display(mse)

rmse = math.sqrt(mse)

rmse



6.383742648831409

2.526606943873821